# performance on sub-graph with smry labels with alternative mapping

In [1]:
# run this only once!!
import os
os. chdir("../..")
os.getcwd()

'/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments'

In [2]:
import networkx as nx
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper
from graphcase_experiments.tools.embedding_plotter import plot_embedding2, plotly_embedding
from graphcase_experiments.tools.calculate_embed import calculate_graphcase_embedding
import matplotlib.pyplot as plt
import pydot

/Users/tonpoppe/opt/anaconda3/envs/graphcase/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load graph
SOURCE_PATH = 'graphcase_experiments/graphs/enron/data/'
G = nx.read_gpickle(SOURCE_PATH + 'enron_sub_graph.pickle')

In [4]:
main_label = {
    'ceos-president': 'senior_mngt',
    'managing director': 'senior_mngt',
    'executive vp': 'senior_mngt',
    'vice president': 'middle_mngt',
    'director': 'middle_mngt',
    'manager': 'middle_mngt',
    'trader': 'trader',
    'specialist': 'employee',
    'employee': 'employee',
    'attorney-legal asst.': 'employee',
}
G2 = G.copy()
relabel =  {n: main_label[a.lower()] for n,a in G.nodes(data='label')}
nx.set_node_attributes(G2, relabel, 'label')

In [5]:
algo = GraphCaseWrapper
embed_gc2, tbl_gc2 = calculate_graphcase_embedding(
    G2, algo.LOCATION, params=algo.COMP_PARAMS, verbose=False, algo=algo
)
plotly_embedding(tbl_gc2)

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-07-15 11:32:13.687767: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-15 11:34:52.531206: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments/graphcase_experiments/tools/embedding_plotter.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['embed1'] = embed[:,0]
/Users/tonpoppe/workspace/graphcas

# performance - sub-graph - smry labels

In [ ]:
%%capture
import mlflow
from graphcase_experiments.experiments.enron_comp import calc_enron_performance
_, smry_df = calc_enron_performance(G=G2, test_size=0.5, runs=5)
smry_df

In [7]:
smry_df

ami                f1_macro            \
                                mean           std      mean       std   
algo                                                                     
DGI                         0.075871  1.455521e-02  0.288071  0.054154   
Drne                        0.106291  2.474560e-03  0.348732  0.029010   
GraphCASE                   0.118823  2.203668e-02  0.443431  0.016346   
MultiLENS                   0.010889  0.000000e+00  0.188461  0.045152   
baseline                    0.073897  3.142117e-03  0.317187  0.059481   
role2vec                    0.048020  2.034863e-02  0.335288  0.037598   
xnetmf                      0.035191  1.003446e-02  0.226922  0.075052   
xnetmf_with_transformation  0.094247  1.387779e-17  0.295520  0.056601   

                            f1_micro            
                                mean       std  
algo                                            
DGI                         0.360526  0.065526  
Drne                        0.439474  0.023903  
GraphCASE                   0.460526  0.016115  
MultiLENS                   0.386842  0.007207  
baseline                    0.392105  0.032763  
role2vec                    0.392000  0.052999  
xnetmf                      0.360526  0.035551  
xnetmf_with_transformation  0.415789  0.035551


# perf support size 15


In [16]:
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper
import tensorflow as tf

class alt_gc(GraphCaseWrapper):
    ENRON_PARAMS = {
        'batch_size': 30,
        'hub0_feature_with_neighb_dim': 128,
        'verbose': False,
        'seed': 1,
        'encoder_labels': ['attr1', 'attr2'],
        'learning_rate': 0.0003668872396300966,
        'act': tf.nn.sigmoid,
        'useBN': True,
        'dropout': 0.09859650451427784,
        'support_size': [15, 15],
        'dims': [3, 128, 128, 128],
        'epochs': 500,
    }

_, smry_df2 = calc_enron_performance(G=G2, algos=[alt_gc], test_size=0.5, runs=5)

/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments/graphcase_experiments/experiments/enron_comp.py:58: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [17]:
smry_df2

ami            f1_macro            f1_micro         
               mean       std      mean       std      mean      std
algo                                                                
GraphCASE  0.153535  0.013607  0.394283  0.043237  0.434211  0.04364

# perf 3 layers

In [18]:
class alt_gc2(GraphCaseWrapper):
    ENRON_PARAMS = {
        'batch_size': 30,
        'hub0_feature_with_neighb_dim': 128,
        'verbose': False,
        'seed': 1,
        'encoder_labels': ['attr1', 'attr2'],
        'learning_rate': 0.0003668872396300966,
        'act': tf.nn.sigmoid,
        'useBN': True,
        'dropout': 0.09859650451427784,
        'support_size': [7,7,7],
        'dims': [3, 128, 128, 128, 128, 128],
        'epochs': 500,
    }

_, smry_df3 = calc_enron_performance(G=G2, algos=[alt_gc2], test_size=0.5, runs=5)

/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments/graphcase_experiments/experiments/enron_comp.py:58: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [20]:
smry_df3

ami           f1_macro            f1_micro          
               mean      std      mean       std      mean       std
algo                                                                
GraphCASE  0.107288  0.01503  0.435792  0.023878  0.460526  0.024616

# learning rate

In [23]:

class alt_gc3(GraphCaseWrapper):
    ENRON_PARAMS = {
        'batch_size': 30,
        'hub0_feature_with_neighb_dim': 128,
        'verbose': False,
        'seed': 1,
        'encoder_labels': ['attr1', 'attr2'],
        'learning_rate': 0.0001,
        'act': tf.nn.sigmoid,
        'useBN': True,
        'dropout': 0.09859650451427784,
        'support_size': [7, 7],
        'dims': [3, 128, 128, 128],
        'epochs': 500,
    }

_, smry_df4 = calc_enron_performance(G=G2, algos=[alt_gc3], test_size=0.5, runs=5)

/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments/graphcase_experiments/experiments/enron_comp.py:58: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [24]:
smry_df4

ami           f1_macro            f1_micro          
               mean       std     mean       std      mean       std
algo                                                                
GraphCASE  0.122114  0.015103  0.42625  0.048947  0.436842  0.049583

In [25]:
class alt_gc3(GraphCaseWrapper):
    ENRON_PARAMS = {
        'batch_size': 30,
        'hub0_feature_with_neighb_dim': 128,
        'verbose': False,
        'seed': 1,
        'encoder_labels': ['attr1', 'attr2'],
        'learning_rate': 0.007,
        'act': tf.nn.sigmoid,
        'useBN': True,
        'dropout': 0.09859650451427784,
        'support_size': [7, 7],
        'dims': [3, 128, 128, 128],
        'epochs': 1000,
    }

_, smry_df5 = calc_enron_performance(G=G2, algos=[alt_gc3], test_size=0.5, runs=5)

In [ ]:
smry_df5